## Implementation of AlexNet

### Building the model

In [21]:
import tensorflow as tf
from tensorflow import keras

In [22]:
def buildModel(numClasses, seed=2019):
    
    poolZ = 3
    poolS = 2
    
    inputs1 = keras.layers.Input(shape=(227, 227, 3), dtype=tf.float32)
    
    # conv1: 96 kernels 11x11x3, stride=4, padding=valid
    x = keras.layers.Conv2D(96, 11, strides=(4, 4), padding='valid', activation=None,
                            input_shape=(227, 227, 3), name='conv1')(inputs1)
    #TODO: response normalization?!
    # maxpool1: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool1')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu1')(x)
    
    # conv2: 256 kernels 5x5x96, stride=1, padding=same
    x = keras.layers.Conv2D(256, 5, padding='same', activation=None, name='conv2')(x)
    #TODO: response normalization?!
    # maxpool2: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool2')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu2')(x)
    
    # conv3: 384 kernels 3x3x256
    x = keras.layers.Conv2D(384, 3, padding='same', activation=None, name='conv3')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu3')(x)
    
    # conv4: 384 kernels 3x3x384
    x = keras.layers.Conv2D(384, 3, padding='same', activation=None, name='conv4')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu4')(x)    
    
    # conv5: 256 kernels 3x3x384
    x = keras.layers.Conv2D(256, 3, padding='same', activation=None, name='conv5')(x)
    # maxpool5: s=2, z=3
    x = keras.layers.MaxPooling2D((poolZ, poolZ), strides=(poolS, poolS), name='maxpool5')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='relu5')(x)
    
    # automatic flattening...
    x = keras.layers.Flatten('channels_last', name='flatten')(x)
    
    # fc1: 4096 units, dropout 0.5
    x = keras.layers.Dense(4096, activation=None, name='fc1')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='fc1_relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x)
    
    # fc2: 4096 units, dropout 0.5
    x = keras.layers.Dense(4096, activation=None, name='fc2')(x)
    x = keras.layers.BatchNormalization(axis=-1)(x)
    x = keras.layers.Activation('relu', name='fc2_relu')(x)
    x = keras.layers.Dropout(rate=0.5)(x) 
    
    # fc3: numClasses units
    x = keras.layers.Dense(numClasses, activation='softmax', name='fc3')(x)
    
    outputs = x
    
    return keras.models.Model(inputs = inputs1, outputs = outputs)

### Reading the dataset

In [23]:
import os
import pandas as pd

In [24]:
def readMyDataset(directory):
    # make a list of all image files
    #imgDirectory = os.path.join(directory, 'processed', 'img')
    #listFiles = [os.path.abspath(f) for f in os.scandir(imgDirectory)]
    #print("Found {} image files.".format(len(listFiles)))
    
    descrFile = os.path.join(directory, 'processed', 'fileLabels.txt')
    
    #pathnames = []
    #labelStrings = []
    df = {'pathnames': [], 'labelStrings': []}
    # make a list of all the files we'll process and their labels
    for line in open(descrFile, 'r'):
        #print(line)
        (fileName, label) = line.split(' ')
        # remove quotes and make absolute path
        fileName = os.path.abspath(os.path.join(directory, 'processed', 'img', fileName[1:-1]))
        #print("{} -> {}".format(fileName, label))
        df['pathnames'].append(fileName)
        # remove trailing newline
        df['labelStrings'].append(label[0:-1])
        
    print("Read description of {} files ({} labels).".format(len(df['pathnames']), len(df['labelStrings'])))
    
    #print("ddddd\n{}\ndddddd".format(df['labelStrings'][0]))
    
    df = pd.DataFrame({'pathname': df['pathnames'], 'labelString': df['labelStrings']})
    print("DataFrame: {} rows.".format(len(df.index)))
    
    dummyLabels = pd.get_dummies(df['labelString'], prefix='', prefix_sep='')
    numClasses = len(dummyLabels.columns)
    classes = list(dummyLabels.columns.values)
    print("number of classes: {}".format(numClasses))
    
    dset = tf.data.Dataset.from_tensor_slices((df['pathname'], dummyLabels))
    
    # function to read image data
    def _howToRead(pathname, label):
        imageData = tf.cast(tf.image.resize_images(
            tf.image.decode_png(tf.read_file(pathname), channels=3), [227, 227]), tf.float32)
        return (imageData, label)
    
    def _howToNormalize(img, label):
        return (img / 255.0, label)
    
    def _howToAugRot90(img, label):
        return (tf.image.rot90(img, k=1), label)
    
    def _howToAugRot180(img, label):
        return (tf.image.rot90(img, k=2), label)
    
    def _howToAugRot270(img, label):
        return (tf.image.rot90(img, k=3), label)
    
    def _howToRandomModify(img, label):
        img = tf.image.random_hue(img, 0.1)
        img = tf.image.random_brightness(img, 0.2)
        return (img, label)
    
    dset = dset.map(_howToRead)
    dsetRandomModified = dset.map(_howToRandomModify)
    dset = dset.map(_howToNormalize).concatenate(dsetRandomModified.map(_howToNormalize)).concatenate(
        dsetRandomModified.map(_howToAugRot90).map(_howToNormalize)).concatenate(
        dsetRandomModified.map(_howToAugRot180).map(_howToNormalize)).concatenate(
        dsetRandomModified.map(_howToAugRot270).map(_howToNormalize))
    
    return (dset, numClasses, classes)
    
    #print("dummies:\n{}".format(dummyLabels))

In [25]:
dataset, numClasses, classes = readMyDataset("/home/ivan/datasets/bilbao2019")

Read description of 1749 files (1749 labels).
DataFrame: 1749 rows.
number of classes: 17


In [26]:
print("{}".format(dataset))

<DatasetV1Adapter shapes: ((227, 227, 3), (17,)), types: (tf.float32, tf.uint8)>


In [27]:
print("Classes: {}".format(classes))

Classes: ['ad', 'bench', 'container', 'door', 'extension', 'gate', 'logo', 'manhole', 'socket', 'staircase', 'switch', 'tap', 'textad', 'toilet', 'urinal', 'urn', 'washbasin']


In [55]:
# just to check if everything is ok

iterator = dataset.make_one_shot_iterator()
nextElement = iterator.get_next()

with tf.Session() as sess:
    cnt = 0
    while True:
        try:
            imgbin, label = sess.run(nextElement)
            #print("{}\n".format(imgbin))
            print("Label # {}: {}".format(cnt, label))
            cnt += 1
        except tf.errors.OutOfRangeError:
            break
    

Label # 0: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 2: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 3: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 4: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 5: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 6: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 7: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 8: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 9: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 10: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 11: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 12: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 13: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 14: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 15: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 16: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 17: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 18: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 19: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 20: [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
La

Label # 199: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 200: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 201: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 202: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 203: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 204: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 205: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 206: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
Label # 207: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 208: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 209: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 210: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 211: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 212: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 213: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 214: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 215: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 216: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 217: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 218: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
Label # 219: [0 0 0 

Label # 406: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 407: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
Label # 408: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 409: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 410: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 411: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 412: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 413: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 414: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 415: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 416: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 417: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 418: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 419: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 420: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 421: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 422: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 423: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 424: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 425: [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
Label # 426: [0 0 0 

Label # 611: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 612: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 613: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 614: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 615: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 616: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 617: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 618: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 619: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 620: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 621: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 622: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 623: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 624: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 625: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 626: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 627: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 628: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 629: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 630: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
Label # 631: [0 0 0 

Label # 806: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 807: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 808: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 809: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 810: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 811: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 812: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 813: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 814: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 815: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 816: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 817: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 818: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 819: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 820: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 821: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 822: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 823: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 824: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 825: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 826: [0 0 1 

Label # 1007: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1008: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1009: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1010: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1011: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1012: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1013: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1014: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1015: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1016: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1017: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1018: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1019: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1020: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1021: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1022: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1023: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1024: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1025: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Label # 1026: [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


Label # 1212: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1213: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1214: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1215: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1216: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1217: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1218: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1219: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1220: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1221: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1222: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1223: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1224: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1225: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1226: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1227: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1228: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1229: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1230: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label # 1231: [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Inputs and outputs

In [28]:
dataset = dataset.shuffle(buffer_size=1000).repeat().batch(32).prefetch(1)

In [12]:
# just to check if everything is ok

#iterator = dataset.make_one_shot_iterator()
#nextElement = iterator.get_next()
#
#with tf.Session() as sess:
#    cnt = 0
#    while True:
#        try:
#            imgbin, label = sess.run(nextElement)
#            #print("{}\n".format(imgbin))
#            print("Label # {}: {}".format(cnt, label))
#            cnt += 1
#        except tf.errors.OutOfRangeError:
#            break

In [29]:
iterator = dataset.make_one_shot_iterator()
inputs, targets = iterator.get_next()
#modelInput = tf.keras.Input(tensor=inputs)
##modelInput = tf.keras.Input(tensor=dataset)

In [30]:
model = buildModel(numClasses)

In [31]:
targets = tf.cast(targets, tf.float32)

In [32]:
model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss='categorical_crossentropy',
             metrics=['accuracy'])#, 
             #target_tensors=targets)

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 55, 55, 96)        34944     
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_v1_7 (Ba (None, 27, 27, 96)        384       
_________________________________________________________________
relu1 (Activation)           (None, 27, 27, 96)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 27, 27, 256)       614656    
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 13, 13, 256)       0         
__________

In [ ]:
#model.fit(epochs=10, steps_per_epoch=50)
#model.fit(epochs=2, steps_per_epoch=50)
##model.fit(epochs=2, steps_per_epoch=50)
model.fit(inputs, targets, epochs=2, steps_per_epoch=200)

Epoch 1/2
200/200 [==============================] - 801s 4s/step - loss: 1.3444 - acc: 0.6259
Epoch 2/2
 14/200 [=>............................] - ETA: 12:29 - loss: 1.4188 - acc: 0.6228

### Save model and weights

In [169]:
modelJson = model.to_json()
with open("alexnet_bilbao2019_new1.json", "w") as jsonFile:
    jsonFile.write(modelJson)

In [170]:
#model.save_weights("alexnet_bilbao2019_weights.h5")
model.save_weights("alexnet_bilbao2019_weights_new1.h5")

### Recover weights

In [34]:
model.load_weights("alexnet_bilbao2019_weights_new1.h5")

### Try with some photos

In [123]:
import numpy as np


def tryOnPhoto(model, photoPathname):
    photo = tf.image.decode_jpeg(tf.read_file(photoPathname), channels=3)
    photo = tf.image.resize_images(photo, (227, 227)) / 255.0
    
    modelInput = None
    photoBin = None
    with tf.Session() as sess:
        photoBin = sess.run(photo)
    
    photoBin = np.expand_dims(photoBin, axis=0)
    print(photoBin.shape)
    
    photoBin = tf.constant((photoBin), shape = (1, 227, 227, 3))
    print(photoBin)
    photoBin = tf.data.Dataset.from_tensor_slices(photoBin).batch(1).repeat()
    print(photoBin)
    iterator = photoBin.make_one_shot_iterator()    
    inputs = iterator.get_next()
    #modelInput = keras.layers.Input(tensor=inputs, shape=(1, 227, 227, 3))
        
    print(modelInput)
    return model.predict(inputs, batch_size=1, steps=1, verbose=1)
    #return model.predict(photoBin)

In [166]:
#testRes = tryOnPhoto(model, "the-fake-downing-street-10-door.jpg")
#testRes = tryOnPhoto(model, "blue-door-london.jpg")
#testRes = tryOnPhoto(model, "Joe-s-Doors-256.jpg")
#testRes = tryOnPhoto(model, "contenedores_basura-6-1-1068x712.jpg")
#testRes = tryOnPhoto(model, "1506704365_421985_1506704486_noticia_normal.jpg")
#testRes = tryOnPhoto(model, "staircase_sf1.jpeg")
#testRes = tryOnPhoto(model, "/home/ivan/datasets/bilbao2019/raw/logo/IMG_20190403_094234.jpg")
testRes = tryOnPhoto(model, "56910-600-338.jpg")
#inputs, targets = iterator.get_next()
#testRes = model.predict(inputs, batch_size=1, steps=1)

(1, 227, 227, 3)
Tensor("Const_31:0", shape=(1, 227, 227, 3), dtype=float32)
<DatasetV1Adapter shapes: (?, 227, 227, 3), types: tf.float32>
None
1/1 [==============================] - 0s 321ms/step


In [167]:
testRes.shape

(1, 17)

In [168]:
print("class: {}, score: {}".format(classes[np.argmax(testRes[0])], np.max(testRes[0])))

class: staircase, score: 0.32549235224723816
